In [ ]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
provider = IBMQ.load_account()

# Chapter 8 - Terra

In [ ]:
# Import registers
from qiskit import QuantumRegister, ClassicalRegister

qr = QuantumRegister(2, 'my_QR')
cr = ClassicalRegister(2, 'my_CR')
qc = QuantumCircuit(qr,cr)
qc.draw()


In [ ]:
qc = QuantumCircuit(QuantumRegister(2, 'my_QR'), ClassicalRegister(2, 'my_CR'))

In [ ]:
#Import the register classes
from qiskit import QuantumRegister, ClassicalRegister
#Create the quantum and classical registers, each with labels
qr1 = QuantumRegister(2, name='qr1')
cr1 = ClassicalRegister(2, name='cr1')
#Create the quantum circuit using the registers
qc1 = QuantumCircuit(qr1, cr1)
#Draw the circuit
qc1.draw()


In [ ]:
#Create two Quantum and Classical registers
qr2 = QuantumRegister(2, name='qr2')
cr2 = ClassicalRegister(2, name='cr2')
#Create a second circuit using the registers created above
qc2 = QuantumCircuit(qr2, cr2)
#Draw the second quantum circuit
qc2.draw()


In [ ]:
#Concatenate the two previous circuits to create a new circuit
#Create an empty quantum circuit
qc_combined = QuantumCircuit()
#Add the two previous quantum and classical registers to the empty quantum circuit
qc_combined.add_register(qr1, qr2, cr1, cr2)
#Draw the concatenated circuit
qc_combined.draw()


In [ ]:
#Import the random_circuit class
from qiskit.circuit.random import random_circuit
#Construct the random circuit with the number of qubits = 3
#with a depth = 2, and include the measurement operator for each qubit
qc = random_circuit(3, 2, measure=True)
#Draw the circuit
qc.draw()


In [ ]:
# Import the random circuit class
from qiskit.circuit.random import random_circuit
#Create two random circuits, each with 2 qubit registers and random #gate operator counts.
qc1 = random_circuit(2,2)
qc2 = random_circuit(2,4)
#Concatenate the two random circuits
qc = qc1.compose(qc2, [0,1])
#Draw the circuit
qc.draw()


In [ ]:
#Define function to print circuit properties:
def print_circuit_props(qc):
    width = qc.width()
    depth = qc.depth()
    num_operators = qc.count_ops()
    circuit_size = qc.size()
    print('Width = ',width) 
    print('Depth = ', depth)
    print('Circuit size = ',circuit_size)
    print('Number of operators = ', num_operators)


In [ ]:
#Pass our quantum circuit to print out the circuit properties
print_circuit_props(qc)


In [ ]:
#Use measure_all() to automatically add the barrier, measurement, and #classical register to our existing circuit.
qc.measure_all()
#Draw the circuit
qc.draw()


In [ ]:
#Print out the circuit properties
print_circuit_props(qc)


In [ ]:
qc = QuantumCircuit(3)
qc.ccx(0,1,2)
qc.draw()


In [ ]:
#Print out the circuit properties
print_circuit_props(qc)


In [ ]:
#Print out the circuit properties
print_circuit_props(qc.decompose())


# Customization and Parameterization of circuits

In [ ]:
#Create a custom two-qubit composite gate
#Create the quantum register
qr = QuantumRegister(2, name='qr_c')
#Generate quantum circuit which will make up the composite gate
comp_qc = QuantumCircuit(qr, name='My-composite')
#Add any gates you wish to your composite gate
comp_qc.h(0)
comp_qc.cx(0, 1)
#Create the composite instructions by converting 
#the QuantumCircuit to a list of Instructions
composite_inst = comp_qc.to_instruction()
#Draw the circuit which will represent the composite gate
comp_qc.draw()


In [ ]:
#Create your 2-qubit circuit to generate your composite gate
qr2 = QuantumRegister(3, 'qr')
#Create a quantum circuit using the quantum register
qc = QuantumCircuit(qr2)
#Add any arbitrary gates that would represent the function 
#of the composite gate 
qc.h(0)
qc.cx(0,1)
qc.cx(0,2)
#Draw the composite circuit
qc.draw()


In [ ]:
#Append your composite gate to the specified qubits. 
qc.append(composite_inst, [qr2[0], qr2[1]])
#Draw the complete circuit
qc.draw()


In [ ]:
#Import the Parameter object
from qiskit.circuit import Parameter
#Construct the Parameter set to Theta
param_theta = Parameter('θ')
#Create a two-qubit quantum circuit and add some gates
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
#Include a rotation gate which we wish to apply 
#the Parameter value 
qc.rz(param_theta,0)
qc.rz(param_theta,1)
#Draw the circuit
qc.draw()


In [ ]:
import numpy as np
#Bind the parameters with a value, in this case 2π
qc = qc.bind_parameters({param_theta: 2*np.pi})
#Draw the circuit with the set parameter values
qc.draw()


# Generating Pulse schedules on quantum hardware

In [ ]:
#Import pulse classes
from qiskit.pulse import Waveform, DriveChannel, Play, Schedule
#Import some helpful utils
from qiskit.scheduler.utils import measure_all


In [ ]:
#Import numpy and generate the sin sample values
import numpy as np
x = np.linspace(0,2*np.pi,64)
data = np.sin(x)
#Generate our sample waveform 
waveform = Waveform(data, name="sin_64_pulse")
#Draw the generated sample waveform
waveform.draw()


In [ ]:
#Import the Gaussian Square pulse from Pulse Library
from qiskit.pulse.library import GaussianSquare
#Create a Gaussian Square pulse: 
#Args: duration, amp, sigma, width, name
gaussian_square_pulse = GaussianSquare(128,1,2,112, name="gaussian square")
gaussian_square_pulse.draw()


# Generating and executing Schedules

In [ ]:
#Create the first schedule with our Gaussian Square pulse
schedule_1 = Schedule(name='Schedule 1')
schedule_1 = schedule_1.insert(0, Play(gaussian_square_pulse, DriveChannel(0)))
#Draw the schedule
schedule_1.draw()


In [ ]:
#Create a second schedule with our sample waveform
schedule_2 = Schedule(name='Schedule 2')
schedule_2 = schedule_2.insert(0, Play(waveform, DriveChannel(0)))
#Draw the schedule
schedule_2.draw()


In [ ]:
#Let's create a third schedule 
#Where we add the first schedule and second schedules
#And shift the second to the right by a time of 5 after the first
schedule_3 = schedule_1.insert(schedule_1.duration+5, schedule_2)
schedule_3.draw()


In [ ]:
#We could have also combined the two using the append operator
#The two schedules are appended immediately after one another
schedule_3_append = schedule_1.append(schedule_2)
schedule_3_append.draw()


# Scheduling existing quantum circuits

In [ ]:
qc = QuantumCircuit(1, 1)
qc.h(0)
qc.measure(0,0)
#Draw the circuit
qc.draw()


In [ ]:
#Import transpile and schedule
from qiskit import transpile, schedule 
#Set the backend to ibmq_armonk
backend = provider.get_backend('ibmq_armonk')
#Transpile the circuit using basis gates from the specified backend
transpiled_qc = transpile(qc, backend)  
#Draw the transpiled circuit
transpiled_qc.draw()


In [ ]:
#Create the circuit schedule using the transpiled circuit
circuit_schedule = schedule(transpiled_qc, backend)
#Draw the circuit
circuit_schedule.draw()


In [ ]:
#Draw the circuit with a shorter time range to ease visibility
circuit_schedule.draw(time_range=[0, 1500])


In [ ]:
#Create a 2-qubit circuit
qc2 = QuantumCircuit(2, 2)
#Apply a Hadamard to the first qubit
qc2.h(0)
#Apply a CNOT gate where the Source is qubit 0, and Target qubit 1
qc2.cx(0, 1)
#Add measurement gates to all qubits
qc2.measure([0, 1], [0, 1])
#Draw the circuit
qc2.draw()


In [ ]:
#Import the test backend
from qiskit.test.mock import FakeAlmaden
#Construct the backend
backend = FakeAlmaden()
#Transpile the circuit to the test backend and its basis states
transpiled_qc2 = transpile(qc2, backend)
#Draw the transpiled circuit
transpiled_qc2.draw()


In [ ]:
#Create the circuit from the transpiled circuit results
circuit_schedule2 = schedule(transpiled_qc2, backend)
#Draw the 2-qubit circuit schedule with range of 2000 time steps
circuit_schedule2.draw(time_range=[0, 2000])


# Leveraging Provider information

In [ ]:
#Import the IBMQ interface
from qiskit import IBMQ

#Save account ONLY needed if running on a local system for first time
#Uncomment below if running for the first time on a local machine.
#IBMQ.save_account('API_TOKEN') 

#Load the account which was saved on local system using save_account.
#Note: this is handled each time a Qiskit Notebook is loaded on IQX.
IBMQ.load_account()


In [ ]:
#Indicate a hub to link account to:
IBMQ.get_provider(hub='ibm-q')
#Indicate a project which your account is associated with
IBMQ.get_provider(project='main')


In [ ]:
#Create the Provider object using the IBMQ interface 
provider = IBMQ.get_provider(group='open')
#Query the list of backends available to your account
provider.backends()


In [ ]:
#Filter the list of backends to include only non-simulator, 
#and operational (meaning, not offline or under maintenance)
provider.backends(simulator=False, operational=True)


In [ ]:
#Select a specific device from the provider
backend = provider.get_backend('ibmq_santiago')


In [ ]:
#Import the least_busy function
from qiskit.providers.ibmq import least_busy

#Identify the least busy devices 
#smaller than 2 qubits and not a simulator
small_devices = provider.backends(filters=lambda x: x.configuration().n_qubits < 2 and not x.configuration().simulator)

#Identify the least busy devices 
#larger than 4 qubits and not a simulator
large_devices = provider.backends(filters=lambda x: x.configuration().n_qubits > 4 and not x.configuration().simulator)

#Print the least busy devices
print('The least busy small devices: {}'.format(least_busy(small_devices)))
print('The least busy large devices: {}'.format(least_busy(large_devices)))


# Quantum backend components

In [ ]:
#Set ibmq_valencia as the backend, or whichever backend you wish
backend = provider.get_backend('ibmq_lima')
#Confirm this is the backend selected by querying for its name,
backend.name()


In [ ]:
#View the status of the backend
status = backend.status()
is_operational = status.operational
jobs_in_queue = status.pending_jobs
print('Number of pending jobs in the queue: ', jobs_in_queue)


In [ ]:
#View the configuration of the backend
backend.configuration()
backend


In [ ]:
# Display the number of qubits from the backend properties
backend.properties().qubits


In [ ]:
#Print out the frequency of qubit (0)
print('Frequency of first qubit is: '+ str(backend.properties().frequency(0)))
#Print out the readout error of qubit (0)
print('Readout error of first qubit is: '+ str(backend.properties().readout_error(0)))


In [ ]:
#Run a few jobs on this backend to generate jobs on the backend
qc = QuantumCircuit(1)
qc.h(0)
qc.measure_all()
qc.draw()

In [ ]:
# Launch the job watcher widget
%qiskit_job_watcher

job_list = []
for i in range(0,1):
    job = execute(qc, backend, shots=1024)
    job_list.append(job)
    result = job.result()
    #result = execute(qc, backend, shots=1024).result()

In [ ]:
print(job_list)

# Understanding the Job component

In [ ]:
#From the previous output of executed jobs, enter its job id.
job = backend.retrieve_job(executed_job.job_id())


In [ ]:
#Print the job instance status
job.status()


In [ ]:
job.result()

In [ ]:
job.backend()

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table